In [22]:
import Pkg
Pkg.add("JuMP")
Pkg.add("HiGHS")
using JuMP  # open source modelling language
using HiGHS  # interface for the open source HiGHS solver for linear optimization
Pkg.status()  # check what packages we've installed

   Resolving package versions...
  No Changes to `~/Documents/climate/braess/Untitled Folder/Project.toml`
  No Changes to `~/Documents/climate/braess/Untitled Folder/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Documents/climate/braess/Untitled Folder/Project.toml`
  No Changes to `~/Documents/climate/braess/Untitled Folder/Manifest.toml`


Status `~/Documents/climate/braess/Untitled Folder/Project.toml`
  [87dc4568] HiGHS v1.12.0
  [7073ff75] IJulia v1.26.0
⌃ [4076af6c] JuMP v1.23.3
Info Packages marked with ⌃ have new versions available and may be upgradable.


In [35]:
function new_flow_model(params_dict)
    #=
    params_dict: supply, demand, a_b_congestion, a_c_congestion, b_d_congestion, c_d_congestion, b_c_congestion
    returns: JuMP model
    =#
    # define the model
    flow_model = Model();
    # specify the solver
    set_optimizer(flow_model, HiGHS.Optimizer)
    #flow_model = Model(HiGHS.Optimizer);
        
    # variables
    @variable(flow_model, a_b_flow>=0)
    @variable(flow_model, a_c_flow>=0)
    @variable(flow_model, b_d_flow>=0)
    @variable(flow_model, c_d_flow>=0)
    @variable(flow_model, b_c_flow>=0)
    
    @variable(flow_model, a_b_cost>=0)
    @variable(flow_model, a_c_cost>=0)
    @variable(flow_model, b_d_cost>=0)
    @variable(flow_model, c_d_cost>=0)
    @variable(flow_model, b_c_cost>=0)

    # constraints
    ab_cost_const = @constraint(flow_model, a_b_cost == params_dict[:a_b_congestion]*a_b_flow)
    ac_cost_const = @constraint(flow_model, a_c_cost == a_c_flow + params_dict[:a_c_congestion])
    bd_cost_const = @constraint(flow_model, b_d_cost == b_d_flow + params_dict[:b_d_congestion])
    cd_cost_const = @constraint(flow_model, c_d_cost == params_dict[:c_d_congestion]*c_d_flow)
    bc_cost_const = @constraint(flow_model, b_c_cost == b_c_flow + params_dict[:b_c_congestion])
    
    node_b_flow_const = @constraint(flow_model, a_b_flow == b_c_flow + b_d_flow)
    node_c_flow_const = @constraint(flow_model, c_d_flow == b_c_flow + a_c_flow)
    
    equal_cost = @constraint(flow_model, a_b_cost + b_d_cost == a_c_cost + c_d_cost) 
    equal_cost_shortcut = @constraint(flow_model, a_b_cost + b_d_cost == a_b_cost + b_c_cost + c_d_cost) 
    supply_eq_flow = @constraint(flow_model, params_dict[:supply] == a_b_flow + a_c_flow)
    demand_eq_flow = @constraint(flow_model, params_dict[:demand] == b_d_flow + c_d_flow)
    
    # objective function
    @expression(flow_model, objective, a_b_cost+b_d_cost)
    @objective(flow_model, Min, objective)
    
    print(flow_model)
    return flow_model
end

new_flow_model (generic function with 2 methods)

In [24]:
# print
function print_mod(flow_model)
    println("Path 1: ", value.(flow_model[:a_b_cost])+value.(flow_model[:b_d_cost]));
    println("a to b: ", value.(flow_model[:a_b_flow]));
    println("b to d: ", value.(flow_model[:b_d_flow]));
    println("Path 2: ", value.(flow_model[:a_c_cost])+value.(flow_model[:c_d_cost]));
    println("a to c: ", value.(flow_model[:a_c_flow]));
    println("c to d: ", value.(flow_model[:c_d_flow]));
    println("Path 3: ", value.(flow_model[:a_b_cost])+value.(flow_model[:b_c_cost])+value.(flow_model[:c_d_cost]));
    println("a to b: ", value.(flow_model[:a_b_flow]));
    println("b to c: ", value.(flow_model[:b_c_flow]));
    println("c to d: ", value.(flow_model[:c_d_flow]));
end

print_mod (generic function with 1 method)

In [36]:
# parameters
param_dict_1 = Dict(
    :supply => 6,
    :demand => 6,
    :a_b_congestion => 10,  # linear
    :a_c_congestion => 50,  # constant
    :b_d_congestion => 50,  # constant
    :c_d_congestion => 10,  # linear
    :b_c_congestion => 10  # constant
)
flow_mod = new_flow_model(param_dict_1)
optimize!(flow_mod)
print_mod(flow_mod)

Running HiGHS 1.8.0 (git hash: fcfb53414): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+01]
  Cost   [1e+00, 1e+00]
  Bound  [0e+00, 0e+00]
  RHS    [6e+00, 5e+01]
Presolving model
4 rows, 3 cols, 10 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-11); columns 0(-10); elements 0(-27) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  9.2000000000e+01
HiGHS run time      :          0.01
Path 1: 92.0
a to b: 4.0
b to d: 2.000000000000007
Path 2: 92.0
a to c: 2.0
c to d: 4.000000000000001
Path 3: 92.0
a to b: 4.0
b to c: 2.0
c to d: 4.000000000000001


In [37]:
# parameters
param_dict_2 = Dict(
    :supply => 6,
    :demand => 6,
    :a_b_congestion => 10,  # linear
    :a_c_congestion => 50,  # constant
    :b_d_congestion => 50,  # constant
    :c_d_congestion => 10,  # linear
    :b_c_congestion => 20  # constant
)

flow_mod_2 = new_flow_model(param_dict_2)
optimize!(flow_mod_2)
print_mod(flow_mod_2)

Running HiGHS 1.8.0 (git hash: fcfb53414): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+01]
  Cost   [1e+00, 1e+00]
  Bound  [0e+00, 0e+00]
  RHS    [6e+00, 5e+01]
Presolving model
4 rows, 3 cols, 10 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-11); columns 0(-10); elements 0(-27) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  8.5076923077e+01
HiGHS run time      :          0.00
Path 1: 85.07692307692308
a to b: 3.230769230769231
b to d: 2.7692307692307736
Path 2: 85.07692307692308
a to c: 2.769230769230769
c to d: 3.2307692307692313
Path 3: 85.07692307692308
a to b: 3.230769230769231
b to c: 0.4615384615384599
c to d: 3.2307692307692313
